In [2]:
import cadquery as cq
workplane = cq.Workplane("XY")

In [3]:
from box_generator.config import BoxConfig

import yaml

with open ("config.yml", "r") as file:
    config = yaml.safe_load(file)

box_config = BoxConfig.load(config)

In [5]:
class Component:
    width: float = 0
    height: float = 0
    depth: float = 0

    x: float = 0
    y: float = 0
    z: float = 0

    rotation: float = 0

    def __init__(self, box_config: BoxConfig):
        pass

class RaspberryPi5(Component):
    def __init__(self):
        self.width = 85.6
        self.height = 17.0
        self.depth = 56.5
    
    def apply(self, box: cq.Workplane, box_config: BoxConfig) -> cq.Workplane:
        cuts = cq.Workplane("XY").box(1,2,1).translate([12,12,12])

        return box.union(cuts)


origin = cq.Workplane("XY").sphere(10)

x = cq.importers.importStep("RP 5.stp")
display(x)

In [7]:
shell = cq.Workplane("XY").box(box_config.width, box_config.length, box_config.height)
inner_cut = (cq.Workplane("XY")
             .box(box_config.width - 2 * box_config.wall_thickness, box_config.length - 2 * box_config.wall_thickness, box_config.height)
             .translate([0,0,box_config.floor_thickness]))
box = shell.cut(inner_cut)

rpi = RaspberryPi5()

box = rpi.apply(box, box_config)
box = box.union(x)
display(box)


cq.exporters.export(box, 'output.stl')